In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy.random as rnd

import matplotlib.animation as animation
from IPython.display import HTML
import ipywidgets as widgets
from IPython.display import display

###############

L=5
r = .5
v0 = 0
nparticulas = 1000
dt=.01

#############




buttonrunsim=widgets.Button(description="Simular")
outwdt = widgets.Output()

radioparticula = widgets.FloatSlider(.1*L,min=.01*L,max=.5*L,step=.1*L,
    description='radio particula',
    ensure_option=True,
    disabled=False)

masaparticula = widgets.FloatLogSlider(value=50, min=.1,max=5,step=.2,
    description='masa de la partícula',
    ensure_option=True,
    disabled=False)


v0particula = widgets.FloatSlider(value=7, min=-10, max=10,step=.1,
    description='velocidad inicial',
    ensure_option=True,
    disabled=False)

fuerzaexterna = widgets.FloatSlider(value=0,min=-100,max=100,step=1.,
    description='Fuerza externa',
    ensure_option=True,
    disabled=False)




masamoleculas = widgets.FloatLogSlider(value=.02,min=-3,max=3,step=.2,
    description='masa de las moléculas',
    ensure_option=True,
    disabled=False)

nmoleculas = widgets.IntSlider(value=1000, min=10, max=5000, step=5,
    description='número de moléculas',
    ensure_option=True,
    disabled=False)

vfluido = widgets.FloatSlider(value=0, min=-10, max=10, step=.1,
    description='velocidad inicial',
    ensure_option=True,
    disabled=False)


temperaturamols = widgets.FloatLogSlider(value=16,min=-1,max=4,step=.2,
    description='temperatura',
    ensure_option=True,
    disabled=False)




items = [[widgets.Label("Partícula"),radioparticula,masaparticula,v0particula,fuerzaexterna,
          widgets.Label("Fluído"),masamoleculas,vfluido,nmoleculas,temperaturamols,buttonrunsim],[outwdt]]

widgets.HBox([widgets.VBox(it) for it in items])

In [2]:
# Inicializar variables


def inicializar():
    global x0, v0, xs, vs, v0, fext, m, mmol, r, masared, fuerzaexterna
    global radioparticula, masaparticula, v0particula, vfluido
    global masamoleculas, nmoleculas, temperaturamols
    r = radioparticula.value
    m = masaparticula.value
    mmol = masamoleculas.value
    fext = fuerzaexterna.value
    masared = 1/(1/m+1/mmol)
    x0 = np.array([0,0])
    v0 = np.array([v0particula.value, 0])
    xs = L*(rnd.uniform(size=(nmoleculas.value,2,))-.5)
    vs = rnd.normal(scale=np.sqrt( masamoleculas.value * temperaturamols.value),
                    size=(nmoleculas.value,2,))
    xs = np.array([x for x in xs if (x[0]**2 + x[1]**2) >1.5*r**2 ])
    vs = vs[0:len(xs)]
    vs[:, 0] = vs[:, 0] + vfluido.value
    fig, ax = plt.subplots(1,2,figsize=(2*L,L))
    particle = ax[0].add_artist(plt.Circle((0,0),r,color="red",fill=True))
    molecules = ax[0].scatter(xs[:,0],xs[:,1])
    plotvxav, = ax[1].plot([0.],[v0[0]])
    return fig, ax, particle, molecules, plotvxav



def paso_dinmol():
    global xs,ps,x0,v0,dt, masared, m, mmol 
    global vfluido, temperaturamols
    xs = xs + vs * dt 
    x0 = x0 + v0 * dt
    x0 = ( x0 + .5*L) % L - .5*L
    dptotal = np.array([0.,0.])
    for i, vmol in enumerate(vs):
        if xs[i,0] > .5*L or xs[i,0] < -.5*L:
            vs[i] = rnd.normal(scale=np.sqrt( masamoleculas.value * temperaturamols.value), size=(2,))
            vs[i][0] = vs[i][0] + vfluido.value
            if vs[i][0]>0:
                xs[i,0] = -.5*L
            else:
                xs[i,0] = .5*L
            continue
        if xs[i,1] > .5*L or xs[i,1] < -.5*L:
            vs[i] = rnd.normal(scale=np.sqrt( masamoleculas.value * temperaturamols.value), size=(2,))
            vs[i][0] = vs[i][0] + vfluido.value
            if vs[i][1]>0:
                xs[i,1] = -.5*L
            else:
                xs[i,1] = .5*L
            continue
        
        xx = xs[i]-x0
        d = np.sqrt(xx.dot(xx)) 
        if d < r:
            xx = xx/d
            deltap = -2.*masared*vmol.dot(xx) *xx  
            dptotal = dptotal - deltap
            vs[i] = vs[i] + deltap /mmol
            xs[i] = xs[i] + 2*dt*vs[i]
    v0 = v0 + dptotal / m
    v0[0] = v0[0] + (fext/m * dt)
    
    

def make_animation_new(ts=300, progress=None):

    fig, ax, particle, molecules, plotvxav = inicializar() 
    ax[1].set_xlim(0,ts)
    ax[1].set_ylim(-10,10)
    
    def update_graphic(t,particle, molecules, plotvxav):
        global xs, x0
        if progress is not None:
            progress.value = t
        for i in range(10):
            paso_dinmol()        
        molecules.set_offsets(xs)
        particle.set_center((x0[0],x0[1]))
        data = plotvxav.get_data()
        plotvxav.set_data(np.append(data[0],data[0][-1]+1),
                             np.append(data[1],v0[0]))
        return particle, molecules, plotvxav

    line_ani = animation.FuncAnimation(fig, update_graphic, ts, fargs=(particle, molecules,plotvxav,),
                                       interval=100, blit=True)
    
    line_ani.save("relajacion.gif", writer="pillow")
    plt.close()
    return HTML(line_ani.to_jshtml())


def on_button_start_sim(b):
    outwdt.clear_output()
    progress = widgets.FloatProgress(value=0,min=0,max=50,step=1,description='Simulating:',
                                         bar_style='info',orientation='horizontal')
    with outwdt:
        display(progress)
        result = make_animation_new(progress=progress)                                          
        #outwdt.clear_output()
        display(result)
        

buttonrunsim.on_click(on_button_start_sim)
